 ## pip install arch prerequired 

In [1]:
cd C:\Users\USER\수정용_금융리스크관리\VaR_and_ES_With_Historical_Simulation_FHS_WHS\Historical_Simulation

C:\Users\USER\수정용_금융리스크관리\VaR_and_ES_With_Historical_Simulation_FHS_WHS\Historical_Simulation


In [2]:
import numpy as np
import pandas as pd
from arch import arch_model
import warnings
import tqdm
warnings.filterwarnings(action = 'ignore')

In [3]:
Price = pd.read_excel('코스피지수.xlsx', parse_dates= ['Date']).set_index('Date')
my_ret = Price.pct_change().rename( columns = {Price.columns[0] : '코스피수익률'}).iloc[1:]
ret = my_ret[my_ret.columns[0]].dropna()

 ### ARCH11
 $$
\begin{array}{c}
\sigma_t^2 = \omega + \alpha_1 u_{t-1}^2
\\
\hat{z}_{t-\tau} = \frac{R_{t-\tau}}{\sigma_{t-\tau}} 
\end{array}
$$
 ### GARCH11
 $$
\begin{array}{c}
\sigma_t^2 = \omega + \alpha u_{t}^2 + \beta \sigma_{t-1}^2
\\
\hat{z}_{t-\tau} = \frac{R_{t-\tau}}{\sigma_{t-\tau}} 
\end{array}
$$


In [4]:
class VaR_FHS :
    
    def ARCH11(ret,alpha = 0.01, max_lag = 252*3) :
        q = 1-alpha
        am1 = arch_model(ret[-max_lag:], p=1, q=0) 
        res1 = am1.fit(disp = 0)
        Z = ret/res1.conditional_volatility
        R = pd.Series(Z * res1.conditional_volatility[-1])
        VaR = R.quantile(1-q)
        ES = R[R<=VaR].mean()
        return {"VaR":VaR, "ES":ES}
    
    def GARCH11(ret, alpha = 0.01, max_lag = 252*3) :
        q = 1-alpha
        gam11 = arch_model(ret[-max_lag:], p=1, q=1)
        resg11 = gam11.fit()
        Z = ret/resg11.conditional_volatility
        R = pd.Series(Z * resg11.conditional_volatility[-1])
        VaR = R.quantile(1-q)
        ES = R[R<=VaR].mean()
        return {"VaR":VaR, "ES":ES}

In [5]:
VaR_FHS.ARCH11(ret,alpha = 0.01, max_lag = 252*3)

Iteration:      1,   Func. Count:      5,   Neg. LLF: 359258.43653791526
Iteration:      2,   Func. Count:     14,   Neg. LLF: -105.40991572788168
Iteration:      3,   Func. Count:     21,   Neg. LLF: -190.04402869161714
Iteration:      4,   Func. Count:     25,   Neg. LLF: 28261.23719155631
Iteration:      5,   Func. Count:     30,   Neg. LLF: -144.8408227235272
Iteration:      6,   Func. Count:     35,   Neg. LLF: 98786.89500195759
Iteration:      7,   Func. Count:     40,   Neg. LLF: -191.7577157191356
Iteration:      8,   Func. Count:     44,   Neg. LLF: -191.7679119252778
Iteration:      9,   Func. Count:     48,   Neg. LLF: -191.76807965502948
Iteration:     10,   Func. Count:     52,   Neg. LLF: -191.76808110194315
Iteration:     11,   Func. Count:     55,   Neg. LLF: -191.76808110198493


{'VaR': -0.12201607518312652, 'ES': -0.17704560903705147}

In [6]:
VaR_FHS.GARCH11(ret,alpha = 0.01, max_lag = 252*3)

Iteration:      1,   Func. Count:      6,   Neg. LLF: 120262801070960.6
Iteration:      2,   Func. Count:     16,   Neg. LLF: 296834921663.1905
Iteration:      3,   Func. Count:     24,   Neg. LLF: -213.40546007304596
Optimization terminated successfully    (Exit mode 0)
            Current function value: -213.40546181354762
            Iterations: 7
            Function evaluations: 24
            Gradient evaluations: 3


{'VaR': -0.07660381478857012, 'ES': -0.09285053257470324}

 ### Weighted Historical Simulation
 $$
\begin{array}{c}
\omega_{\tau} = \frac{1-\lambda}{1-\lambda^m} \lambda^{\tau-1} \\ 
(if \ \tau \ is \ 1, \ the \ most \ recent \ wegiht ) \\ 
( m \ is \ total \ observation )
\end{array}
$$


In [7]:
def VaR_WHS(ret, alpha = 0.01, lamb = 0.97) :
    q = 1-alpha
    m = len(ret)
    ret = pd.Series(ret)
    tau = np.arange(1,m+1)
    w = pd.Series((lamb**(tau-1) * (1-lamb) / (1-lamb**m))[::-1], index = pd.Series(ret).index)
    data = pd.concat( [ret, w], axis = 1)
    data.columns = ['ret','w']
    sorted_data = data.sort_values(by  = [data.columns[0]])
    sorted_data['cum_w'] = sorted_data.w.cumsum(0)
    n = np.abs(np.array(sorted_data.cum_w) - (1-q)).argmin()
    my_data = sorted_data.iloc[n-1:n+2].set_index('cum_w')['ret']
    temp = pd.Series([np.nan], index = [1-q])
    VaR = pd.concat([temp,my_data], axis = 0).sort_index().interpolate(method = 'linear').loc[1-q]
    ES = ret[ret<=VaR].mean()
    return {'VaR':VaR, 'ES':ES}

In [8]:
VaR_WHS(ret, alpha = 0.01, lamb = 0.97)

{'VaR': -0.07270484035952518, 'ES': -0.1250012516988628}